### Day 10 - Part A
Button combinations to turn the light on

In [162]:
import copy
import math
import sympy as sym
import numbers
from itertools import product
from itertools import chain, combinations
from util.aoc_utility import *

DAY = 10
TEST_MODE = 0
SECTIONS = False

data_in = load_input(day=DAY, test_mode=TEST_MODE, sections=SECTIONS)
data_pc = [line_to_arr(x, delimiter="]") for x in data_in]

setup = []

for line in data_in:

    #Get light pattern
    light_end = line.find("]")
    light_section = line[1:light_end]
    light_section_num = []
    for char in light_section:
        if char == ".":
            light_section_num.append(0)
        else:
            light_section_num.append(1)

    #Buttons
    button_end = line.find("{")
    button_section = line[light_end+3:button_end].split("(")
    buttons = []
    for button in button_section:
        button_nums = button[:-2].split(",")
        buttons.append([int(x) for x in button_nums])

    #Joltage
    joltage_section = line[button_end+1:-1]
    joltage_nums = [int(x) for x in joltage_section.split(",")]
    
    config = {"lights":light_section_num, "buttons": buttons, "joltages":joltage_nums}
    setup.append(config)

In [ ]:
def positive_integer_assignments(n, limits):
   all_values = [list(values) for values in product(range(0, max(limits)+1), repeat=n)]
   combos = [x for x in all_values if all([x[idx] <= limits[idx] for idx in range(len(limits))])]
   return combos

def get_eqs(state, buttons):
    
    sol = 999

    syms = sym.symbols(",".join([chr(ord('a')+i) for i in range(len(buttons))]), integer=True)

    eqs = []
    for idx_s, state_val in enumerate(state):
        vars = []
        for idx_b, button in enumerate(buttons):
            if idx_s in button:
                vars.append(syms[idx_b])

        eqs.append(sym.Eq(sum(vars), state_val))

    #Determine free variables in advance
    #Preferred the ones with the smallest limits
    free_var_count = len(syms) - len(eqs)

    if free_var_count > 0:
        free_var_candidates = []
        for idx_b, button in enumerate(buttons):
            button_limit = min([state[x] for x in button])
            free_var_candidates.append((idx_b, button_limit))

        free_var_candidates.sort(key=lambda x:x[1], reverse=True)

        lock_vars = [x[0] for x in free_var_candidates]

        syms_vars = [syms[x] for x in lock_vars]

        result = sym.solve(eqs, syms_vars, dict=True)[0]

    else:
        result = sym.solve(eqs, (syms))

    #Get free variables
    free_vars = []
    free_vars_idx = []
    test_vals = {}

    for idx, symbol in enumerate(syms):
        if symbol not in result.keys():
            free_vars.append(symbol)
            free_vars_idx.append(idx)
            test_vals[symbol] = 0

    if not free_vars:
        return sum([result[x] for x in result.keys()])

    free_vars_limits = []

    for idx_b in free_vars_idx:
        button = buttons[idx_b]
        free_vars_max = min([state[x] for x in button])
        free_vars_limits.append(free_vars_max)

    combos = positive_integer_assignments(len(free_vars), free_vars_limits)
    if len(combos) > 1000000:
        print("Help:", len(combos))

    for combo in combos:
        
        for idx, symbol in enumerate(free_vars):
            test_vals[symbol] = combo[idx]

        tested = {var: expr.subs(test_vals) for var, expr in result.items()}

        if all([tested[x] >= 0 for x in tested.keys()]) and all([isinstance(tested[x], numbers.Integral) for x in tested.keys()]):
            sol = min(sol, sum([tested[x] for x in tested.keys()]) + sum(combo))

    return sol
    

In [167]:
total = 0
sols = []
for idx, config in enumerate(setup):
    sol = get_eqs(config["joltages"], config["buttons"])

    print(idx, config["joltages"], sol)
    total += sol
    sols.append(sol)

total

0 [41, 56, 34, 37, 46, 22, 41, 67, 71] 75
578
1 [33, 65, 16, 88, 54, 52] 104
2 [212, 250, 36, 8, 36, 231, 239, 23, 17] 250
3
3 [2, 16, 21, 21, 7] 26
484
4 [58, 25, 73, 46, 65, 21, 44] 94
5 [32, 18, 14, 19, 49, 49] 51
2304
6 [83, 47, 99, 85, 57, 97, 59, 79, 50, 108] 125
7 [14, 14, 13, 33, 20, 34, 22] 55
20
8 [120, 24, 40, 123, 19, 50, 37, 149, 47, 33] 187
9 [25, 29, 19, 17, 25, 37] 54
10 [38, 27, 22, 33, 41, 18, 26, 31, 43, 33] 74
11 [140, 176, 13, 168, 16, 49, 35, 20, 34, 161] 196
12 [31, 50, 169, 29, 212, 50, 188, 55, 50] 229
13 [29, 31, 38, 23] 49
14 [29, 57, 46, 58, 38] 84
64
15 [129, 63, 63, 180, 180, 138, 73, 53, 45] 201
16 [42, 11, 39, 34, 46, 71, 60, 39, 50, 75] 92
17 [49, 48, 36, 51, 81, 33, 71] 88
19604
18 [28, 25, 29, 35, 44, 39, 62, 58, 34, 30] 78
19 [133, 116, 19, 136, 28, 39, 31, 109, 132] 160
34
20 [33, 33, 17, 39, 4, 26] 56
21 [3, 3, 9, 12, 15] 15
31
22 [182, 188, 156, 173, 43, 157, 30, 34, 184, 196] 218
23 [131, 132, 128, 7, 22, 147, 141, 161] 161
676
24 [237, 25, 29, 2

16513